Requires Biovec environment, see clustering notebook.

In [27]:
# mamba create -n biovec python=3.6 ipython pandas ipykernel
# mamba activate biovec
# pip3 install biovec
# make package (might throw some errors since ancient version of python)

# ipython
# import biovec
# pv = biovec.models.ProtVec("swissprot.fasta", corpus_fname="output_corpusfile_path.txt", workers=80)

# import biovec

# pv = biovec.models.ProtVec(
#     "some_fasta_file.fasta", corpus_fname="output_corpusfile_path.txt"
# )

# # The n-gram "QAT" should be trained in advance
# pv["QAT"]

# # convert whole amino acid sequence into vector
# pv.to_vecs("ATATQSQSMTEEL")

# # save trained model into file
# pv.save("model_file_path")

# # load trained model from file
# pv2 = biovec.models.load_protvec("model_file_path")


In [1]:
import pandas as pd

sequences = pd.read_table("/home/ad/biovec_test/uniprot_transporter_sequences.tsv", index_col=0, squeeze=True)

In [2]:
import biovec

In [6]:
pv = biovec.models.load_protvec("/home/ad/biovec_test/swissprot.protvec.model")

In [9]:
import numpy as np
import re

def split_ngrams(seq, n):
    """
    'AGAMQSASM' => [['AGA', 'MQS', 'ASM'], ['GAM','QSA'], ['AMQ', 'SAS']]
    """
    a, b, c = zip(*[iter(seq)]*n), zip(*[iter(seq[1:])]*n), zip(*[iter(seq[2:])]*n)
    str_ngrams = []
    for ngrams in [a,b,c]:
        x = ["".join(ngram) for ngram in ngrams]
        str_ngrams.append(x)
    return str_ngrams

# def generate_corpusfile(fasta_fname, n, corpus_fname):
#     '''
#     Args:
#         fasta_fname: corpus file name
#         n: the number of chunks to split. In other words, "n" for "n-gram"
#         corpus_fname: corpus_fnameput corpus file path
#     Description:
#         Protvec uses word2vec inside, and it requires to load corpus file
#         to generate corpus.
#     '''
#     f = open(corpus_fname, "w")
#     fasta = Fasta(fasta_fname)
#     for record_id in tqdm(fasta.keys(), desc='corpus generation progress'):
#         r = fasta[record_id]
#         seq = str(r)
#         ngram_patterns = split_ngrams(seq, n)
#         for ngram_pattern in ngram_patterns:
#             f.write(" ".join(ngram_pattern) + "\n")
#     f.close()

# def to_vecs(seq, size=100, n=3):
#     """
#     convert sequence to three n-length vectors
#     e.g. 'AGAMQSASM' => [ array([  ... * 100 ], array([  ... * 100 ], array([  ... * 100 ] ]
#     """
#     seq = seq.replace("*","") # for sequences with * to indicate stop
#     ngram_patterns = split_ngrams(seq, n)
#     protvecs = np.zeros([3,size]) # numpy for easy integration with ML libraries
#     for num in range(3):
#         ngram_vecs = []
#         for ngram in ngram_patterns[num]:
#             try:
#                 ngram_vecs.append(pv[ngram])
#             except:
#                 raise Exception("Model has never trained this n-gram: " + ngram)
#         protvecs[num,] = sum(ngram_vecs)
#     return protvecs


In [11]:
from gensim.models import word2vec

model_w2v = word2vec.Word2Vec.load("/home/ad/biovec_test/swissprot.protvec.model")

In [18]:
test_sequence = sequences[0]
test_sequence

'MALQSQASEEAKGPWQEADQEQQEPVGSPEPESEPEPEPEPEPVPVPPPEPQPEPQPLPDPAPLPELEFESERVHEPEPTPTVETRGTARGFQPPEGGFGWVVVFAATWCNGSIFGIHNSVGILYSMLLEEEKEKNRQVEFQAAWVGALAMGMIFFCSPIVSIFTDRLGCRITATAGAAVAFIGLHTSSFTSSLSLRYFTYGILFGCGCSFAFQPSLVILGHYFQRRLGLANGVVSAGSSIFSMSFPFLIRMLGDKIKLAQTFQVLSTFMFVLMLLSLTYRPLLPSSQDTPSKRGVRTLHQRFLAQLRKYFNMRVFRQRTYRIWAFGIAAAALGYFVPYVHLMKYVEEEFSEIKETWVLLVCIGATSGLGRLVSGHISDSIPGLKKIYLQVLSFLLLGLMSMMIPLCRDFGGLIVVCLFLGLCDGFFITIMAPIAFELVGPMQASQAIGYLLGMMALPMIAGPPIAGLLRNCFGDYHVAFYFAGVPPIIGAVILFFVPLMHQRMFKKEQRDSSKDKMLAPDPDPNGELLPGSPNPEEPI'

In [21]:
ngram_vectors = split_ngrams(test_sequence, 3)  # Three vectors, for the three starting points.

In [29]:
ngram_vector1 = ngram_vectors[0]  # kmers in sequence when starting at 0 and sliding by three 

In [31]:
ngram_vector1_ngram1 = ngram_vector1[0]
ngram_vector1_ngram1

'MAL'

In [34]:
ngram_vector1_ngram2 = ngram_vector1[1]
ngram_vector1_ngram2

'QSQ'

In [41]:
model_w2v.wv[ngram_vector1_ngram1]

array([ 1.18808679e-01, -3.64876986e-01,  6.47983193e-01, -1.46426350e-01,
       -1.52609244e-01, -1.66515931e-01, -3.83192301e-01, -1.84395611e-01,
       -2.08178714e-01, -2.30770335e-01, -6.94512352e-02, -4.52766428e-04,
        4.31878418e-02,  1.23070590e-01, -8.74646232e-02, -3.87230255e-02,
        2.65492380e-01,  5.03719375e-02,  3.54006499e-01,  6.21168278e-02,
       -2.57201105e-01, -1.36489691e-02, -1.99244395e-02,  4.79546823e-02,
        1.71690788e-02, -5.52854389e-02,  9.23806354e-02,  7.55709736e-03,
        5.39546832e-03,  7.81023037e-03,  3.50544862e-02, -1.86856706e-02,
       -4.65532169e-02, -1.07445650e-01,  1.13820970e-01, -7.78736696e-02,
       -5.66957742e-02,  7.97994137e-02, -1.19515814e-01,  4.82424311e-02,
        1.71776205e-01, -1.00432143e-01,  2.24059612e-01, -9.02400762e-02,
       -9.13605988e-02,  1.33866414e-01, -1.31028533e-01, -3.13403383e-02,
       -1.58041149e-01, -1.74196381e-02, -1.17266186e-01, -8.67590979e-02,
       -1.10433914e-01, -

In [43]:
sum([model_w2v.wv[ngram_vector1_ngram1], model_w2v.wv[ngram_vector1_ngram2]])

array([-0.13461933, -0.7111297 ,  1.0208125 , -0.30877602, -0.23616222,
       -0.25844306, -0.50489485, -0.08331859, -0.56245625, -0.31021148,
        0.0801694 ,  0.04676345, -0.45370948,  0.3514357 , -0.32759288,
        0.08846295,  0.6903758 , -0.06264126,  0.2731142 ,  0.18539931,
       -0.4407096 , -0.04989775, -0.09461653, -0.11871418,  0.11407162,
       -0.0691107 ,  0.14646347,  0.19423391,  0.14325903, -0.02893887,
        0.10630041, -0.10056195,  0.04517817, -0.27524903,  0.00641009,
       -0.04310326, -0.20991428,  0.27244735, -0.00301873, -0.06236536,
        0.17510992, -0.16946386,  0.46349233, -0.16256922,  0.1544082 ,
        0.3841244 , -0.31079298, -0.08728868, -0.08097062, -0.14748202,
       -0.20406985, -0.13692127, -0.3161991 , -0.23075272,  0.14425012,
       -0.01757712,  0.0135788 ,  0.14987619,  0.2340453 , -0.21841547,
       -0.07607324,  0.12445412, -0.27735892,  0.09093606,  0.13038597,
       -0.06298643,  0.10943196,  0.02477781,  0.06615185, -0.00

Word embeddings are explained here in detail: https://www.youtube.com/watch?v=gQddtTdmG_8 . 

In short, the algorithm is trained on a corpus, i.e. a text. After training, it can produce a vector of length n (for example n=100) for each word. The similarity between two words is then calculated through the cosine distance, which is given as dot(X,Y)/(norm(X)*norm(Y)), where X and Y are the vectors for a given word/ngram/kmer and norm() is the euclidean norm. 

A higher similarity score means that the words are more similar, in the context of the text that was fed to the neural network.

The idea behind the Biovec package is to transform a sequence of amino acids into three vectors of 3-mers. Each starts at a different sequence position (0,1,2). This is probably supposed to teach the kmer order to the algorithm.

Every sequence is then treated as a "sentence", where the words are the kmers in the sequence. One of the standard methods for calculating the similarity between two sentences is to simply add their vectors/embeddings together, and then calculate the cosine distance between the resulting vectors. Here is an example, using normal words and sentences:

In [101]:
# example: from https://stackoverflow.com/a/66127454

from scipy import spatial
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-50") #choose from multiple models https://github.com/RaRe-Technologies/gensim-data

In [108]:
s0 = 'Mark zuckerberg owns the facebook company'
s1 = 'Facebook company ceo is mark zuckerberg'
s2 = 'Microsoft is owned by Bill gates'
s3 = 'How to learn japanese'

def preprocess(s):
    return [i.lower() for i in s.split()]

def get_vector(s):
    return np.sum(np.array([model[i] for i in preprocess(s)]), axis=0)

def get_similarity(v0,v1):
    return 1 - spatial.distance.cosine(v0, v1)

print('s0 vs s1 ->',1 - spatial.distance.cosine(get_vector(s0), get_vector(s1)))
print('s0 vs s2 ->', 1 - spatial.distance.cosine(get_vector(s0), get_vector(s2)))
print('s0 vs s3 ->', 1 - spatial.distance.cosine(get_vector(s0), get_vector(s3)))

s0 vs s1 -> 0.965923011302948
s0 vs s2 -> 0.8659112453460693
s0 vs s3 -> 0.5877998471260071


These models and datasets are from a github repository that lists example datasets. The number at the end stands for the vector size. The example above uses vectors of length 50, the highest number seems to be 300. This seems to be a tradeoff between storage space/computation time, and accuracy.

An alternative that has not been tried before would be to use Doc2Vec, which is available in the same python package (gensim). TODO I should try that. This post explains Doc2Vec, and also a Word2Vec solution that used Cosine similarity and average vectors: https://datascience.stackexchange.com/a/23998

In [116]:
# len(get_vector(s1))

50

Word movers distance algorithm is another method, and is also included in the gensim package: https://datascience.stackexchange.com/a/31497

Finally, another score I came across was the DICE score, which is a variant of the F1 score. I would create a set of kmers, then divide the number of kmers in the intersection set by the total number of kmers. This would not even need any word embeddings.

## Encoding sequences with biovec package:

#### Training (takes a few hours!)

In [ ]:
import biovec

retrain = False
if retrain:
    pv = biovec.models.ProtVec("/home/ad/biovec_test/swissprot.fasta", corpus_fname="/home/ad/biovec_test/output_corpusfile_path.txt", workers=80)
    pv.save('/home/ad/biovec_test/swissprot.protvec.model')
pv = biovec.models.load_protvec('swissprot.protvec.model')

#### Application

In [13]:
def get_nlp_feature(sequence:str):
    arrays = pv.to_vecs(sequence)
    return pd.Series(np.concatenate(arrays))
encoded = sequences.apply(get_nlp_feature)
encoded.to_csv("/home/ad/biovec_test/uniprot_transporter_sequences_encoded.tsv", sep="\t")
encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
Uniprot,,,,,,,,,,,,,,,,,,,,,
P36021,-7.273644,-34.285099,68.556938,-12.244725,-5.441298,-10.215006,-68.510017,-22.670380,-39.208435,-11.681757,...,4.140173,7.689297,-8.704616,-3.346005,2.328844,-5.019221,1.109853,16.568235,20.324194,-6.998810
P43005,-4.140399,-32.438766,63.623219,-14.523251,-6.829222,-10.790448,-71.488716,-19.858337,-35.338825,-8.385507,...,3.039154,9.374497,-13.243600,-3.349254,-1.535459,-2.477201,2.961823,19.141970,21.797276,-5.176438
P43007,-6.950311,-34.846478,67.427254,-14.485823,-6.727311,-10.528156,-72.611015,-21.206213,-38.286049,-10.396502,...,7.083755,6.126781,-10.004407,-1.570559,0.785607,-4.293931,1.667595,14.724697,17.068716,-5.410892
Q5TGU0,-3.028285,-12.994875,22.860323,-1.317772,-4.241808,-4.403067,-20.944134,-6.393130,-10.412485,-1.996316,...,1.936083,0.124781,-2.916309,1.244631,-0.117676,-2.666773,2.721554,3.414848,5.361672,-2.029417
Q92536,-5.437946,-33.504353,63.424423,-9.342864,-9.211921,-9.999936,-67.682892,-19.497953,-34.540096,-8.593810,...,6.935112,7.029308,-11.998138,0.645455,0.094206,-2.755319,4.371674,15.640297,18.742008,-6.791428
